# 기본설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gdown
!pip install torch
!pip install ultralytics

In [ ]:
!gdown https://drive.google.com/drive/folders/1rId2BezmoOPzWj2T0iuyeDrJ3t6Cn4zr?usp=sharing

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/drive/folders/1rId2BezmoOPzWj2T0iuyeDrJ3t6Cn4zr?usp=sharing
To: /content/1rId2BezmoOPzWj2T0iuyeDrJ3t6Cn4zr?usp=sharing
1.19MB [00:00, 97.8MB/s]


# 필요한 라이브러리 로드

In [ ]:
# 기본 라이브러리
import os
import random
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

# 이미지 처리 관련 라이브러리
import cv2
from PIL import Image
from sklearn.preprocessing import LabelEncoder

# PyTorch 관련 라이브러리
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.models import resnet18

# 컴퓨터 비전 모델 관련 라이브러리
from ultralytics import YOLO

# 평가 지표 관련 라이브러리
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 1-1. training, validation 통계표 작성

##  training 통계표

In [4]:
training_folder_path = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/training_image'

# 폴더 내의 모든 파일 목록 중 .jpg 파일만 필터링
training_image_files = [f for f in os.listdir(training_folder_path) if f.endswith('.jpg')]

training_gender_style_list = []

# 파일 이름에서 성별과 스타일 추출
for file in training_image_files:
    parts = file.split('_')
    # 성별 (마지막 부분에서 .jpg 제거)
    gender = parts[-1].replace('.jpg', '')
    style = parts[-2]
    # 리스트에 성별과 스타일 추가
    training_gender_style_list.append((gender, style))

# 성별을 기준으로 정렬
training_gender_style_list = sorted(training_gender_style_list, key=lambda x: x[0])

# 성별 & 스타일 통계 계산
training_counter = Counter(training_gender_style_list)

# DataFrame 생성 (성별, 스타일, 카운트를 별도로 분리)
training_df = pd.DataFrame([(gender, style, count) for (gender, style), count in training_counter.items()],
                           columns=['성별', '스타일', '이미지수'])

# 인덱스를 기준으로 오름차순 정렬
training_df = training_df.sort_index(ascending=True)
training_df

,성별,스타일,이미지수
0,M,metrosexual,278
1,M,normcore,364
2,M,ivy,237
3,M,hiphop,274
4,M,mods,269
5,M,bold,268
6,M,sportivecasual,298
7,M,hippie,260
8,W,powersuit,120
9,W,bodyconscious,95


## validation 통계표

In [3]:
validation_folder_path = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/validation_image'

# 폴더 내의 모든 파일 목록 중 .jpg 파일만 필터링
validation_image_files = [f for f in os.listdir(validation_folder_path) if f.endswith('.jpg')]

validation_gender_style_list = []

# 파일 이름에서 성별과 스타일 추출
for file in validation_image_files:
    parts = file.split('_')
    # 성별 (마지막 부분에서 .jpg 제거)
    gender = parts[-1].replace('.jpg', '')
    style = parts[-2]
    # 리스트에 성별과 스타일 추가
    validation_gender_style_list.append((gender, style))

# 성별을 기준으로 정렬
validation_gender_style_list = sorted(validation_gender_style_list, key=lambda x: x[0])

# 성별 & 스타일 통계 계산
validation_counter = Counter(validation_gender_style_list)

# DataFrame 생성 (성별, 스타일, 카운트를 별도로 분리)
validation_df = pd.DataFrame([(gender, style, count) for (gender, style), count in validation_counter.items()],
                           columns=['성별', '스타일', '이미지수'])

# 인덱스를 기준으로 오름차순 정렬
validation_df = validation_df.sort_index(ascending=True)
validation_df

,성별,스타일,이미지수
0,M,hiphop,66
1,M,normcore,51
2,M,hippie,82
3,M,ivy,79
4,M,bold,57
5,M,mods,80
6,M,metrosexual,58
7,M,sportivecasual,52
8,W,sportivecasual,48
9,W,feminine,44


# 1-2 resnet-18 클래스 분류 및 정확도 제시


## 이미지 전처리(YOLO)

In [ ]:
# 유효한 클래스 목록 정의
valid_classes = {
    0: ('M', 'hippie'),
    1: ('M', 'mods'),
    2: ('M', 'ivy'),
    3: ('M', 'hiphop'),
    4: ('M', 'metrosexual'),
    5: ('M', 'bold'),
    6: ('M', 'sportivecasual'),
    7: ('M', 'normcore'),
    8: ('W', 'sportivecasual'),
    9: ('W', 'feminine'),
    10: ('W', 'minimal'),
    11: ('W', 'powersuit'),
    12: ('W', 'bodyconscious'),
    13: ('W', 'classic'),
    14: ('W', 'kitsch'),
    15: ('W', 'normcore'),
    16: ('W', 'cityglam'),
    17: ('W', 'oriental'),
    18: ('W', 'ecology'),
    19: ('W', 'space'),
    20: ('W', 'athleisure'),
    21: ('W', 'hippie'),
    22: ('W', 'genderless'),
    23: ('W', 'punk'),
    24: ('W', 'grunge'),
    25: ('W', 'disco'),
    26: ('W', 'military'),
    27: ('W', 'hiphop'),
    28: ('W', 'popart'),
    29: ('W', 'lounge'),
    30: ('W', 'lingerie')
}

In [ ]:
# YOLOv8 모델 로드
yolo_model = YOLO('yolov8s.pt')

# 이미지에서 person만 탐지하고 가장 큰 박스만 자르기 함수
def detect_and_crop(image_path, output_folder):
    print(f"Processing file: {image_path}")  # 파일 경로 출력
    try:
        img = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None  # 이미지 로딩 실패 시 None 반환

    img_np = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    results = yolo_model(img_np, classes=[0])  # 사람만 탐지

    max_area = 0
    largest_box = None

    for result in results:
        boxes = result.boxes.xyxy
        classes = result.boxes.cls

        for box, cls in zip(boxes, classes):
            class_name = yolo_model.names[int(cls)]

            if class_name == 'person':
                x1, y1, x2, y2 = map(int, box)
                area = (x2 - x1) * (y2 - y1)

                if area > max_area:
                    max_area = area
                    largest_box = (x1, y1, x2, y2)

    if largest_box:
        x1, y1, x2, y2 = largest_box
        cropped_img = img_np[y1:y2, x1:x2]

        # 파일 이름에서 gender와 style 추출
        parts = os.path.basename(image_path).split('_')
        if len(parts) >= 5:  # 파일명 형식에 맞는지 확인
            gender = parts[4].replace('.jpg', '')  # 'W' 또는 'M'
            style = parts[3]   # 스타일 정보 (예: 'normcore')

            print(f"Extracted gender: {gender}, style: {style}")  # 추출된 gender와 style 출력

            # valid_classes에서 style에 해당하는 인덱스 찾기
            class_index = get_class_index(gender, style)

            if class_index is not None:
                output_path = os.path.join(output_folder, f"cropped_{os.path.basename(image_path).replace('.jpg', '_largest.jpg')}")
                print(f"Saving cropped image to: {output_path}")  # 저장 경로 출력
                cv2.imwrite(output_path, cropped_img)

                return cropped_img
            else:
                print(f"Class ({gender}, {style}) not found in valid_classes")  # 유효하지 않은 클래스일 경우 출력
        else:
            print(f"Filename {os.path.basename(image_path)} is not in the expected format.")
    else:
        print("No person detected or no valid bounding box found")  # 사람 탐지 실패 시 출력

    return None

In [ ]:
# 유효한 클래스 인덱스를 찾는 함수
def get_class_index(gender, style):
    for index, (g, s) in valid_classes.items():
        if g == gender and s == style:
            return index
    return None  # 유효하지 않은 경우 None 반환

# 사용자 정의 데이터셋 클래스
class GenderStyleDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
        self.transform = transform

    def get_label_from_filename(self, filename):
        parts = filename.split('_')
        if len(parts) >= 5:  # Check if the filename format is correct
            gender = parts[5].replace('.jpg', '')  # Extract gender (M or W), remove .jpg
            style = parts[4]   # Extract style
            return f"{gender}_{style}"
        else:
            print(f"Filename {filename} is not in the expected format.")
            return None  # Return None for unexpected formats

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = os.path.join(self.folder_path, image_file)
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            # Return a black image or handle accordingly
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        label = self.get_label_from_filename(image_file)

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 폴더 경로 설정
train_folder_path = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/training_image'
train_output_folder = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/cropped_training_images'

# 이미지 자르기
for filename in os.listdir(train_folder_path):
    if filename.endswith('.jpg'):
        image_path = os.path.join(train_folder_path, filename)
        print(f"Calling detect_and_crop for: {filename}")  # 호출 시 출력
        detect_and_crop(image_path, train_output_folder)

# 데이터셋 및 DataLoader 정의
train_dataset = GenderStyleDataset(train_output_folder, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)

# 레이블 인코딩 + 클래스 상세 확인
label_encoder = LabelEncoder()

# 모든 파일의 레이블을 수집하여 LabelEncoder에 적합
train_labels = [train_dataset.get_label_from_filename(f) for f in train_dataset.image_files if train_dataset.get_label_from_filename(f) is not None]
label_encoder.fit(train_labels)  # 레이블 인코더에 학습

# 클래스 개수 및 각 클래스의 인코딩 값 출력
num_classes = len(label_encoder.classes_)
print(f"Number of classes: {num_classes}")
print("Classes and their encoded values:", dict(zip(label_encoder.classes_, range(num_classes))))

# 인덱스 번호에 맞게 출력
for index in range(num_classes):
    gender, style = label_encoder.classes_[index].split('_')
    class_index = get_class_index(gender, style)
    print(f"{class_index}: {label_encoder.classes_[index]}")

# Validation 폴더 경로 및 출력 경로 설정
valid_folder_path = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/validation_image'
valid_output_folder = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/cropped_validation_images'

# Validation 이미지 자르기
for filename in os.listdir(valid_folder_path):
    if filename.endswith('.jpg'):
        image_path = os.path.join(valid_folder_path, filename)
        print(f"Calling detect_and_crop for validation file: {filename}")  # 호출 시 출력
        detect_and_crop(image_path, valid_output_folder)

# Validation 데이터셋 및 DataLoader 정의
valid_dataset = GenderStyleDataset(valid_output_folder, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False, num_workers=4)

## Modeling

In [ ]:
# Random state 설정
def set_seed(seed=20):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed(20)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 클래스 정의
class GenderStyleDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
        print(f"Loaded {len(self.image_files)} images from {folder_path}")
        self.transform = transform

    def get_label_from_filename(self, filename):
        parts = filename.split('_')
        gender = parts[-2]
        style = parts[-3].replace('.jpg', '')
        return f"{gender}_{style}"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = os.path.join(self.folder_path, image_file)

        # 이미지 열기
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.get_label_from_filename(image_file)

        return image, label

# 경로 설정
train_output_folder = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/cropped_training_images'
val_output_folder = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/cropped_validation_images'

# 학습 데이터셋 및 DataLoader 정의
train_dataset = GenderStyleDataset(train_output_folder, transform=transform)
val_dataset = GenderStyleDataset(val_output_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, drop_last=True)

# 레이블 인코더로 클래스 인덱스 확인
label_encoder = LabelEncoder()
train_labels = [train_dataset.get_label_from_filename(f) for f in train_dataset.image_files if train_dataset.get_label_from_filename(f) is not None]
label_encoder.fit(train_labels)

# 클래스 수 확인
num_classes = len(label_encoder.classes_)
print(f"Number of classes: {num_classes}")
# 클래스 레이블 목록 출력
print("Class labels:", label_encoder.classes_)

Loaded 4063 images from /content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/cropped_training_images
Loaded 943 images from /content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/cropped_validation_images
Number of classes: 31
Class labels: ['M_bold' 'M_hiphop' 'M_hippie' 'M_ivy' 'M_metrosexual' 'M_mods'
 'M_normcore' 'M_sportivecasual' 'W_athleisure' 'W_bodyconscious'
 'W_cityglam' 'W_classic' 'W_disco' 'W_ecology' 'W_feminine'
 'W_genderless' 'W_grunge' 'W_hiphop' 'W_hippie' 'W_kitsch' 'W_lingerie'
 'W_lounge' 'W_military' 'W_minimal' 'W_normcore' 'W_oriental' 'W_popart'
 'W_powersuit' 'W_punk' 'W_space' 'W_sportivecasual']


In [ ]:
# ResNet-18 모델 정의
model = resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Early Stopping을 위한 변수
patience = 5  # 검증 손실 개선을 기다릴 에포크 수
best_val_loss = float('inf')  # 현재까지의 최저 검증 손실
epochs_no_improve = 0  # 개선되지 않은 에포크 수

num_epochs = 20  # 최대 에포크 수
early_stop = False  # 조기 종료 플래그

for epoch in range(num_epochs):
    if early_stop:
        print("조기 종료가 발동되었습니다.")
        break

    # Training phase
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = [label_encoder.transform([label])[0] for label in labels]
        labels = torch.tensor(labels).to(device)

        # 순전파 및 손실 계산
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 손실 기록
        running_loss += loss.item()

        # 예측 결과에서 가장 높은 값의 인덱스를 선택
        _, predicted = torch.max(outputs, 1)

        # 정확도 계산
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

        # 역전파 및 가중치 갱신
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크별 학습 정확도
    train_accuracy = correct_predictions / total_predictions * 100

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0

    val_all_labels = []
    val_all_preds = []

    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images = val_images.to(device)
            val_labels = [label_encoder.transform([label])[0] for label in val_labels]
            val_labels = torch.tensor(val_labels).to(device)

            val_outputs = model(val_images)
            v_loss = criterion(val_outputs, val_labels)
            val_loss += v_loss.item()

            _, val_predicted = torch.max(val_outputs, 1)
            correct_val_predictions += (val_predicted == val_labels).sum().item()
            total_val_predictions += val_labels.size(0)

            val_all_labels.extend(val_labels.cpu().numpy())
            val_all_preds.extend(val_predicted.cpu().numpy())

    val_accuracy = correct_val_predictions / total_val_predictions * 100

    # Validation F1-Score 계산, zero_division=1로 경고 억제
    val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(
        val_all_labels, val_all_preds, average='weighted', zero_division=1
    )

    # 에포크별 손실 및 정확도 출력
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%, "
          f"Val F1-Score: {val_f1:.2f}")

    # Early Stopping 조건 체크
    current_val_loss = val_loss / len(val_loader)

    if current_val_loss < best_val_loss:
        best_val_loss = current_val_loss
        epochs_no_improve = 0  # 개선된 경우 카운트 초기화
        torch.save(model.state_dict(), 'best_resnet18_model.pth')  # 개선될 때마다 모델 저장
    else:
        epochs_no_improve += 1  # 개선되지 않은 경우 카운트 증가

    if epochs_no_improve >= patience:
        print(f"Early stopping 발동! {patience} 에포크 동안 검증 손실이 개선되지 않았습니다.")
        early_stop = True  # 조기 종료 플래그 설정

# 학습 완료 후 F1-Score 확인
if not early_stop:
    torch.save(model.state_dict(), 'trained_resnet18_model.pth')
print("모델 학습이 완료되었습니다.")

Epoch [1/30], Loss: 3.1896, Train Accuracy: 9.10%, Val Loss: 3.0726, Val Accuracy: 9.59%, Val F1-Score: 0.06
Epoch [2/30], Loss: 2.9968, Train Accuracy: 13.37%, Val Loss: 2.9669, Val Accuracy: 11.64%, Val F1-Score: 0.10
Epoch [3/30], Loss: 2.8696, Train Accuracy: 16.96%, Val Loss: 2.8493, Val Accuracy: 16.81%, Val F1-Score: 0.13
Epoch [4/30], Loss: 2.7112, Train Accuracy: 21.08%, Val Loss: 2.7291, Val Accuracy: 18.86%, Val F1-Score: 0.16
Epoch [5/30], Loss: 2.4813, Train Accuracy: 28.47%, Val Loss: 2.8565, Val Accuracy: 17.03%, Val F1-Score: 0.14
Epoch [6/30], Loss: 2.1958, Train Accuracy: 37.50%, Val Loss: 2.8613, Val Accuracy: 20.47%, Val F1-Score: 0.20


In [ ]:
# 모델 가중치 다운로드 (Colab에서 실행 시)
from google.colab import files
files.download('best_resnet18_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>